# Setup

In [ ]:
!pip install sympy

In [ ]:
import copy
import random
import numpy as np
from scipy.interpolate import lagrange
import sys
import itertools
from functools import reduce
from sympy import symbols, Dummy
from sympy.polys.domains import ZZ
from sympy.ntheory.primetest import isprime
from sympy.polys.galoistools import (gf_irreducible_p, gf_add, \
                                     gf_sub, gf_mul, gf_rem, gf_gcdex)

In [ ]:
class GF():
    def __init__(self, p, n=1):
        p, n = int(p), int(n)
        if not isprime(p):
            raise ValueError("p must be a prime number, not %s" % p)
        if n <= 0:
            raise ValueError("n must be a positive integer, not %s" % n)
        self.p = p
        self.n = n
        if n == 1:
            self.reducing = [1, 0]
        else:
            for c in itertools.product(range(p), repeat=n):
              poly = (1, *c)
              if gf_irreducible_p(poly, p, ZZ):
                  self.reducing = poly
                  break

    def add(self, x, y):
        l = gf_add(x, y, self.p, ZZ)
        while len(l) != 6:
          l.insert(0, 0)
        return l

    def sub(self, x, y):
        l = gf_sub(x, y, self.p, ZZ)
        while len(l) != 6:
          l.insert(0, 0)
        return l

    def mul(self, x, y):
        p = gf_rem(gf_mul(x, y, self.p, ZZ), self.reducing, self.p, ZZ)
        while len(p) < 6:
            p.insert(0, 0)
        return p

    def inv(self, x):
        while x[0] == 0:
            x.pop(0)
        s, t, h = gf_gcdex(x, self.reducing, self.p, ZZ)
        while len(s) < 6:
            s.insert(0, 0)
        return s

In [ ]:
def block_separator(message):
    i = 0
    blocks = []
    lista = []
    for m in message:
        if i < 3:
            lista.append(m)
            i += 1
        else:
            lista.append(m)
            blocks.append(lista)
            lista = []
            i = 0
    if lista != []:
        blocks.append(lista)
    while len(blocks[-1]) < 4:
        blocks[-1].insert(3, " ")
    return blocks

def source_coding(abc, word):
  m = []
  for i in range(len(word)):
    m.append(abc[word[i]])
  return m

def f_m(m, x):
    c = 0
    x_power = copy.deepcopy(x)
    for i in range(len(m)):
      if i == 0:
        c = m[0]
      else:
        c = F.add(c, F.mul(m[i], x_power))
        x_power = F.mul(x_power, x)
    return c

def damage(c, error_list):
  c_damaged = copy.deepcopy(c)
  for i in error_list:
    c_damaged[i-1] = []
    for j in range(6):
      c_damaged[i-1].append(random.randint(0, 1))
  return c_damaged

def gauss(alpha, c_damaged):
    n = 8
    a = []

    for i in range(n):
        l = []
        for j in range(n+1):
            l.append(0)
        a.append(l)

    for i in range(n):
        for j in range(n+1):
            if j == 0:
                a[i][j] = idx_poly[1]
            elif j > 0 and j <= 5:
                co = alpha[i]
                for k in range(j-1):
                    co = F.mul(co, alpha[i])
                a[i][j] = co
            elif j == 6:
                a[i][j] = c_damaged[i]
            elif j == 7:
                co = F.mul(alpha[i], c_damaged[i])
                a[i][j] = co
            else:
                co = F.mul(c_damaged[i], F.mul(alpha[i], alpha[i]))
                a[i][j] = co

    n = 8
    number_of_errors = 2

    for i in range(n):
        if a[i][i] == [0, 0, 0, 0, 0, 0] and i != n-1:
            k = 1
            while a[i+k][i] == [0, 0, 0, 0, 0, 0]:
                k += 1
                if i+k > 7:
                    number_of_errors = 0
                    return a, number_of_errors
            temp = copy.deepcopy(a[i])
            a[i] = copy.deepcopy(a[i+k])
            a[i+k] = temp

        if a[i][i] == 1:
            pass
        else:
            divisor = copy.deepcopy(a[i][i])
            for j in range(n+1):
                if divisor == [0, 0, 0, 0, 0, 0]:
                    pass
                else:
                    inverse = F.inv(divisor)
                    a[i][j] = F.mul(a[i][j], inverse)

        for j in range(n):
            if j == i:
                pass
            else:
                ratio = a[j][i]
                for k in range(i, n+1):
                    s = F.mul(ratio, a[i][k])
                    a[j][k] = F.sub(a[j][k], s)

    if a[-1][-2] == [0, 0, 0, 0, 0, 0]:
        number_of_errors = 1

        n = 8
        a = []

        for i in range(n):
            l = []
            for j in range(n-1):
                l.append(0)
            a.append(l)

        for i in range(n):
            for j in range(n-1):
                if j == 0:
                    a[i][j] = idx_poly[1]
                elif j > 0 and j <= 4:
                    co = alpha[i]
                    for k in range(j-1):
                        co = F.mul(co, alpha[i])
                    a[i][j] = co
                elif j == 5:
                    a[i][j] = c_damaged[i]
                else:
                    co = F.mul(alpha[i], c_damaged[i])
                    a[i][j] = co

        n = 6

        for i in range(n):
            if a[i][i] == [0, 0, 0, 0, 0, 0] and i != n-1:
                k = 1
                while a[i+k][i] == [0, 0, 0, 0, 0, 0]:
                    k += 1
                temp = copy.deepcopy(a[i])
                a[i] = copy.deepcopy(a[i+k])
                a[i+k] = temp

            if a[i][i] == 1:
                pass
            else:
                divisor = copy.deepcopy(a[i][i])
                for j in range(n+1):
                    if divisor == [0, 0, 0, 0, 0, 0]:
                        pass
                    else:
                        inverse = F.inv(divisor)
                        a[i][j] = F.mul(a[i][j], inverse)

            for j in range(n+2):
                if j == i:
                    pass
                else:
                    ratio = a[j][i]
                    for k in range(i, n+1):
                        s = F.mul(ratio, a[i][k])
                        a[j][k] = F.sub(a[j][k], s)

    return a, number_of_errors

def error_locator(a, numer_of_errors):
    l = []
    if number_of_errors == 0:
        return l
    elif number_of_errors == 1:
        if a[-3][-1] == [0, 0, 0, 0, 0, 0]:
            return l
        else:
            l.append(a[-3][-1])
            return l
    else:
        l1 = a[6][8]
        l2 = a[7][8]
        for i in alpha:
            s = F.sub(l2, i)
            if l1 == F.mul(s, i):
                e2 = i
        e1 = F.sub(l2, e2)
        l.append(e1)
        l.append(e2)
        return l

def decoder(alpha, c_damaged, number_of_errors, damaged_alpha, idx_poly):

    alpha_lack = list(copy.deepcopy(alpha))
    c_lack = copy.deepcopy(c_damaged)
    ind = []
    mes = []
    message_back = str()
    abc_key_list = list(abc.keys())
    abc_val_list = list(abc.values())

    for e in damaged_alpha:
        alpha_lack.remove(e)

    for e in damaged_alpha:
        i = alpha.index(e)
        ind.append(i)

    ind.sort(reverse=True)

    for i in ind:
        c_lack.pop(i)

    n = 4
    a = []

    for i in range(n):
        l = []
        for j in range(n+1):
            l.append(0)
        a.append(l)

    for i in range(n):
        for j in range(n+1):
            if j == 0:
                a[i][j] = idx_poly[1]
            elif j == 1:
                a[i][j] = alpha_lack[i]
            elif j == 2:
                pw = F.mul(alpha_lack[i], alpha_lack[i])
                a[i][j] = pw
            elif j == 3:
                pw = F.mul(alpha_lack[i], alpha_lack[i])
                cu = F.mul(pw, alpha_lack[i])
                a[i][j] = cu
            else:
                a[i][j] = c_lack[i]

    n = 4

    for i in range(n):
        if a[i][i] == [0, 0, 0, 0, 0, 0]:
            k = 1
            while a[i+k][i] == [0, 0, 0, 0, 0, 0]:
                k += 1
            temp = copy.deepcopy(a[i])
            a[i] = copy.deepcopy(a[i+k])
            a[i+k] = temp

        if a[i][i] == [0, 0, 0, 0, 0, 1]:
            pass
        else:
            divisor = copy.deepcopy(a[i][i])
            for j in range(n+1):
                inverse = F.inv(divisor)
                a[i][j] = F.mul(a[i][j], inverse)

        for j in range(n):
            if j == i:
                pass
            else:
                ratio = a[j][i]
                for k in range(i, n+1):
                    s = F.mul(ratio, a[i][k])
                    a[j][k] = F.sub(a[j][k], s)

    for i in range(len(a)):
        mes.append(a[i][-1])

    mes[0] = F.mul(mes[0], [0, 0, 0, 0, 1, 0])

    for m in mes:
        message_back = message_back + abc_key_list[abc_val_list.index(m)]

    return message_back

def block_concatenator(blocks):
    message = str()
    for block in blocks:
        message = message + block
    return message

# Input

In [ ]:
F = GF(2,6)

In [ ]:
abc = {' ':[0, 0, 0, 0, 0, 0],
       'a':[0, 0, 0, 0, 1, 0],
       'b':[0, 0, 0, 1, 0, 0],
       'c':[0, 0, 1, 0, 0, 0],
       'd':[0, 1, 0, 0, 0, 0],
       'e':[1, 0, 0, 0, 0, 0],
       'f':[0, 0, 0, 0, 1, 1],
       'g':[0, 0, 0, 1, 1, 0],
       'h':[0, 0, 1, 1, 0, 0],
       'i':[0, 1, 1, 0, 0, 0],
       'j':[1, 1, 0, 0, 0, 0],
       'k':[1, 0, 0, 0, 1, 1],
       'l':[0, 0, 0, 1, 0, 1],
       'm':[0, 0, 1, 0, 1, 0],
       'n':[0, 1, 0, 1, 0, 0],
       'o':[1, 0, 1, 0, 0, 0],
       'p':[0, 1, 0, 0, 1, 1],
       'q':[1, 0, 0, 1, 1, 0],
       'r':[0, 0, 1, 1, 1, 1],
       's':[0, 1, 1, 1, 1, 0],
       't':[1, 1, 1, 1, 0, 0],
       'u':[1, 1, 1, 0, 1, 1],
       'v':[1, 1, 0, 1, 0, 1],
       'w':[1, 0, 1, 0, 0, 1],
       'x':[0, 1, 0, 0, 0, 1],
       'y':[1, 0, 0, 0, 1, 0],
       'z':[0, 0, 0, 1, 1, 1],
       'A':[0, 0, 1, 1, 1, 0],
       'B':[0, 1, 1, 1, 0, 0],
       'C':[1, 1, 1, 0, 0, 0],
       'D':[1, 1, 0, 0, 1, 1],
       'E':[1, 0, 0, 1, 0, 1],
       'F':[0, 0, 1, 0, 0, 1],
       'G':[0, 1, 0, 0, 1, 0],
       'H':[1, 0, 0, 1, 0, 0],
       'I':[0, 0, 1, 0, 1, 1],
       'J':[0, 1, 0, 1, 1, 0],
       'K':[1, 0, 1, 1, 0, 0],
       'L':[0, 1, 1, 0, 1, 1],
       'M':[1, 1, 0, 1, 1, 0],
       'N':[1, 0, 1, 1, 1, 1],
       'O':[0, 1, 1, 1, 0, 1],
       'P':[1, 1, 1, 0, 1, 0],
       'Q':[1, 1, 0, 1, 1, 1],
       'R':[1, 0, 1, 1, 0, 1],
       'S':[0, 1, 1, 0, 0, 1],
       'T':[1, 1, 0, 0, 1, 0],
       'U':[1, 0, 0, 1, 1, 1],
       'V':[0, 0, 1, 1, 0, 1],
       'W':[0, 1, 1, 0, 1, 0],
       'X':[1, 1, 0, 1, 0, 0],
       'Y':[1, 0, 1, 0, 1, 1],
       'Z':[0, 1, 0, 1, 0, 1],
       '=':[1, 0, 1, 0, 1, 0],
       '!':[0, 1, 0, 1, 1, 1],
       '?':[1, 0, 1, 1, 1, 0],
       '.':[0, 1, 1, 1, 1, 1],
       ',':[1, 1, 1, 1, 1, 0],
       ';':[1, 1, 1, 1, 1, 1],
       '+':[1, 1, 1, 1, 0, 1],
       '-':[1, 1, 1, 0, 0, 1],
       '*':[1, 1, 0, 0, 0, 1],
       '/':[1, 0, 0, 0, 0, 1],
       '%':[0, 0, 0, 0, 0, 1]
       }

idx_poly = {0:[0, 0, 0, 0, 0, 0],
            1:[0, 0, 0, 0, 1, 0],
            2:[0, 0, 0, 1, 0, 0],
            3:[0, 0, 1, 0, 0, 0],
            4:[0, 1, 0, 0, 0, 0],
            5:[1, 0, 0, 0, 0, 0],
            6:[0, 0, 0, 0, 1, 1],
            7:[0, 0, 0, 1, 1, 0],
            8:[0, 0, 1, 1, 0, 0],
            9:[0, 1, 1, 0, 0, 0],
            10:[1, 1, 0, 0, 0, 0],
            11:[1, 0, 0, 0, 1, 1],
            12:[0, 0, 0, 1, 0, 1],
            13:[0, 0, 1, 0, 1, 0],
            14:[0, 1, 0, 1, 0, 0],
            15:[1, 0, 1, 0, 0, 0],
            16:[0, 1, 0, 0, 1, 1],
            17:[1, 0, 0, 1, 1, 0],
            18:[0, 0, 1, 1, 1, 1],
            19:[0, 1, 1, 1, 1, 0],
            20:[1, 1, 1, 1, 0, 0],
            21:[1, 1, 1, 0, 1, 1],
            22:[1, 1, 0, 1, 0, 1],
            23:[1, 0, 1, 0, 0, 1],
            24:[0, 1, 0, 0, 0, 1],
            25:[1, 0, 0, 0, 1, 0],
            26:[0, 0, 0, 1, 1, 1],
            27:[0, 0, 1, 1, 1, 0],
            28:[0, 1, 1, 1, 0, 0],
            29:[1, 1, 1, 0, 0, 0],
            30:[1, 1, 0, 0, 1, 1],
            31:[1, 0, 0, 1, 0, 1],
            32:[0, 0, 1, 0, 0, 1],
            33:[0, 1, 0, 0, 1, 0],
            34:[1, 0, 0, 1, 0, 0],
            35:[0, 0, 1, 0, 1, 1],
            36:[0, 1, 0, 1, 1, 0],
            37:[1, 0, 1, 1, 0, 0],
            38:[0, 1, 1, 0, 1, 1],
            39:[1, 1, 0, 1, 1, 0],
            40:[1, 0, 1, 1, 1, 1],
            41:[0, 1, 1, 1, 0, 1],
            42:[1, 1, 1, 0, 1, 0],
            43:[1, 1, 0, 1, 1, 1],
            44:[1, 0, 1, 1, 0, 1],
            45:[0, 1, 1, 0, 0, 1],
            46:[1, 1, 0, 0, 1, 0],
            47:[1, 0, 0, 1, 1, 1],
            48:[0, 0, 1, 1, 0, 1],
            49:[0, 1, 1, 0, 1, 0],
            50:[1, 1, 0, 1, 0, 0],
            51:[1, 0, 1, 0, 1, 1],
            52:[0, 1, 0, 1, 0, 1],
            53:[1, 0, 1, 0, 1, 0],
            54:[0, 1, 0, 1, 1, 1],
            55:[1, 0, 1, 1, 1, 0],
            56:[0, 1, 1, 1, 1, 1],
            57:[1, 1, 1, 1, 1, 0],
            58:[1, 1, 1, 1, 1, 1],
            59:[1, 1, 1, 1, 0, 1],
            60:[1, 1, 1, 0, 0, 1],
            61:[1, 1, 0, 0, 0, 1],
            62:[1, 0, 0, 0, 0, 1],
            63:[0, 0, 0, 0, 0, 1]
          }

alpha = [[0, 1, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 1],
         [1, 1, 1, 1, 0, 0],
         [0, 1, 1, 1, 0, 0],
         [0, 1, 0, 1, 1, 0],
         [1, 0, 1, 1, 0, 1],
         [0, 1, 0, 1, 0, 1],
         [1, 1, 1, 0, 0, 1]]

In [ ]:
message = "The cat is sleeping. The dog is eating."

# Encoding

In [ ]:
c_list = []

block_list = block_separator(message)

for block in block_list:
    c = []
    m = source_coding(abc, block)
    for a in alpha:
        c.append(f_m(m, a))
    c_list.append(c)

print("First element in c_list: ", c_list[0])

# Channel

In [ ]:
c_damaged_list = []

for c in c_list:
    t = random.randint(0,2)
    error_list = random.sample(range(1, len(alpha)+1), t)
    c_damaged = damage(c, error_list)
    c_damaged_list.append(c_damaged)

print("First element in c_damaged_list: ", c_damaged_list[0])

# Decoding

In [ ]:
decoded_message_list = []

for c in c_damaged_list:
    A, number_of_errors = gauss(alpha, c)
    damaged_alpha = error_locator(A, number_of_errors)
    decoded_message_list.append(decoder(alpha, c, number_of_errors, damaged_alpha, idx_poly))

print("The original message was: ", block_concatenator(decoded_message_list))